In [9]:
import torch
import torch.optim as optim
from torchvision.models import resnet18
import torch.nn as nn
from kvasir_capsule_dataset import get_dataloader
from trainer import train_kc_model
import gc

with_gpu = torch.cuda.is_available()

if with_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print('We are now using %s.' % device)

train_path = r"C:\Users\JadHa\Desktop\Uni\DLMI-Project\kvasir-capsule-labeled-images\dataset_train.csv"
val_path = r"C:\Users\JadHa\Desktop\Uni\DLMI-Project\kvasir-capsule-labeled-images\dataset_test.csv"
dataset_path = r"C:\Users\JadHa\Desktop\Uni\DLMI-Project\kvasir-capsule-labeled-images\labelled_images"

resnet = resnet18(num_classes=2).to(device)

train_loader = get_dataloader(csv_path=train_path, dataset_path=dataset_path, batch_size=64, shuffle=True)
val_loader = get_dataloader(csv_path=val_path, dataset_path=dataset_path, batch_size=64, shuffle=True)

optimizer = optim.Adam(params=resnet.parameters(), lr=3e-4)

criterion = nn.CrossEntropyLoss()

We are now using cuda.


In [10]:
torch.cuda.empty_cache()
gc.collect()
train_kc_model(resnet, optimizer, criterion, train_loader, val_loader, epochs=10, device=device)

Metrics for the last batch :
Epoch : 1, Loss : 0.00, Accuracy : 1.00, Precision : No Data, Recall : No Data
Metrics for the last batch :
Epoch : 1, Loss : 0.40, Accuracy : 0.94, Precision : 0.00, Recall : 0.00
